In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet, MobileNetV2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [175]:
class VeggieVision:
    def __init__(self, base_model, input_shape=(224, 224, 3)):
        # Set parameters
        self.input_shape = input_shape
        self.base_model = base_model
        self.model = None
        self.datagen = None

        # Build the model
        self.build_model()

    def build_model(self):
        # Freeze the base model layers initially
        self.base_model.trainable = False

       # 2. Build the Model
        self.model = models.Sequential()

        # Add the base model (MobileNetV2)
        self.model.add(self.base_model)

        self.model.add(layers.Flatten())

        # Add an additional Dense layer with 128 neurons and ReLU activation
        self.model.add(layers.Dense(1024, activation='relu'))
        self.model.add(layers.Dropout(0.2))


        #   Final Dense layer with a single neuron for regression (predicting weight)
        self.model.add(layers.Dense(1, activation='linear'))
        

    def augment_data(self, X_train):
        # 5. Data Augmentation
        # Create an ImageDataGenerator for real-time data augmentation
        self.datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            horizontal_flip=True,      # Randomly flip images horizontally
            zoom_range=0.1,            # Zoom in/out by up to 10%
            width_shift_range=0.1,     # Shift the image horizontally by up to 10%
            height_shift_range=0.1,    # Shift the image vertically by up to 10%
            brightness_range=[0.8, 1.2],  # Random brightness between 80% and 120%
            shear_range=0.1,           # Small shearing for slight perspective shifts
            fill_mode='nearest'        # Fill any gaps after transformation with nearest pixels
        )

        # Fit the ImageDataGenerator on the training data
        self.datagen.fit(X_train)

    def fit(self, X, y, batch_size=32, epochs=20, learning_rate=1e-4, validation_split=0.2, fine_tune=False, unfreeze_layers = 20):
        # 4. Split Data into Training and Validation Sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)

        # Augment the data
        self.augment_data(X_train)

        # Compile the model with the provided learning rate
        self.model.compile(optimizer=Adam(learning_rate=learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # 6. Train the Model
        # Use the data generator to augment and train the model
        history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                 validation_data=(X_val, y_val), 
                                 epochs=epochs)

        # Evaluate the model on the validation set
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after initial training: {test_mae}')

        # 8. Fine-tuning the model (if requested)
        if fine_tune:
            self.fine_tune(X_train, y_train, X_val, y_val, batch_size, epochs, learning_rate, unfreeze_layers)

        return history

    def fine_tune(self, X_train, y_train, X_val, y_val, batch_size=32, epochs=10, fine_tune_learning_rate=1e-5, unfreeze_layers=20):
        # Get the total number of layers in the base model
        total_layers = len(self.base_model.layers)

        # Unfreeze the last 'unfreeze_layers' layers of the base model
        for layer in self.base_model.layers[-unfreeze_layers:]:
            layer.trainable = True

        # Keep other layers frozen
        for layer in self.base_model.layers[:-unfreeze_layers]:
            layer.trainable = False

        # Compile with a lower learning rate for fine-tuning
        self.model.compile(optimizer=Adam(learning_rate=fine_tune_learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # Fine-tune the model with augmented data
        fine_tune_history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                           validation_data=(X_val, y_val), 
                                           epochs=epochs)

        # Evaluate after fine-tuning
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after fine-tuning: {test_mae}')

        return fine_tune_history
    
    def evaluate(self, X, y):
        test_loss, test_mae = self.model.evaluate(X, y)
        print(f'MAE: {test_mae}')
        return test_loss, test_mae


In [95]:
# Load the data
X = np.load("../data/processed_data/X.npy")
y = np.load("../data/processed_data/y.npy")

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [176]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_512 = VeggieVision(base_model)

In [177]:
veggie_512.fit(X_train, y_train, learning_rate = 1e-5, fine_tune=True, unfreeze_layers=10)

Epoch 1/20


Exception ignored in: <function _EagerDefinedFunctionDeleter.__del__ at 0x000001EAA1232790>
Traceback (most recent call last):
  File "c:\Users\juliu\.conda\lib\site-packages\tensorflow\python\eager\function.py", line 305, in __del__
    context.remove_function(self.name)
  File "c:\Users\juliu\.conda\lib\site-packages\tensorflow\python\eager\context.py", line 2740, in remove_function
    context().remove_function(name)
KeyboardInterrupt: 


14/15 [===========================>..] - ETA: 0s - loss: 32839.1875 - mae: 102.0216

KeyboardInterrupt: 

In [125]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_512 = VeggieVision(base_model)

In [138]:
veggie_512.fit(X_train, y_train, learning_rate = 1e-5, fine_tune=True, unfreeze_layers=20)

Epoch 1/20
15/15 [==============================] - 12s 717ms/step - loss: 13562.0518 - mae: 24.3591 - val_loss: 595.6191 - val_mae: 18.4901
Epoch 2/20
15/15 [==============================] - 10s 683ms/step - loss: 12716.6133 - mae: 22.9725 - val_loss: 592.7847 - val_mae: 18.4149
Epoch 3/20
15/15 [==============================] - 11s 700ms/step - loss: 12861.3574 - mae: 23.0187 - val_loss: 602.3264 - val_mae: 18.4252
Epoch 4/20
15/15 [==============================] - 11s 697ms/step - loss: 12916.8525 - mae: 22.3725 - val_loss: 598.1050 - val_mae: 18.3796
Epoch 5/20
15/15 [==============================] - 10s 693ms/step - loss: 12755.2578 - mae: 22.8867 - val_loss: 602.1756 - val_mae: 18.3618
Epoch 6/20
15/15 [==============================] - 10s 691ms/step - loss: 12971.7910 - mae: 23.0000 - val_loss: 619.4124 - val_mae: 18.5368
Epoch 7/20
 3/15 [=====>........................] - ETA: 7s - loss: 690.5701 - mae: 18.8646

KeyboardInterrupt: 

In [137]:
veggie_512.evaluate(X_test, y_test)

5/5 [==============================] - 2s 381ms/step - loss: 597.7286 - mae: 18.1945
MAE: 18.194547653198242


(597.7286376953125, 18.194547653198242)

In [111]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_128_32 = VeggieVision(base_model)

In [116]:
veggie_128_32.fit(X_train, y_train, learning_rate = 1e-5, unfreeze_layers= 15)

Epoch 1/20
15/15 [==============================] - 13s 741ms/step - loss: 11198.3154 - mae: 21.8105 - val_loss: 643.3034 - val_mae: 17.5778
Epoch 2/20
15/15 [==============================] - 10s 680ms/step - loss: 11925.3936 - mae: 21.6513 - val_loss: 641.2964 - val_mae: 17.5027
Epoch 3/20
15/15 [==============================] - 10s 684ms/step - loss: 12435.4443 - mae: 21.9274 - val_loss: 648.8666 - val_mae: 17.4616
Epoch 4/20
15/15 [==============================] - 10s 687ms/step - loss: 11106.4082 - mae: 20.5328 - val_loss: 668.0463 - val_mae: 17.6257
Epoch 5/20
 1/15 [=>............................] - ETA: 10s - loss: 423.0233 - mae: 15.1681

KeyboardInterrupt: 

In [117]:
veggie_128_32.evaluate(X_test, y_test)

5/5 [==============================] - 2s 370ms/step - loss: 532.6274 - mae: 17.1669
MAE: 17.166948318481445


(532.6273803710938, 17.166948318481445)

In [71]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_64 = VeggieVision(base_model)

In [77]:
veggie_64.fit(X_train, y_train, learning_rate = 1e-4, fine_tune = True, unfreeze_layers = 15)

Epoch 1/20
15/15 [==============================] - 12s 731ms/step - loss: 13700.1914 - mae: 33.2652 - val_loss: 892.5553 - val_mae: 23.3741
Epoch 2/20
15/15 [==============================] - 11s 709ms/step - loss: 14527.6025 - mae: 33.5551 - val_loss: 890.4384 - val_mae: 23.4008
Epoch 3/20
15/15 [==============================] - 10s 677ms/step - loss: 13744.5928 - mae: 32.6945 - val_loss: 890.9908 - val_mae: 23.4213
Epoch 4/20
 4/15 [=======>......................] - ETA: 6s - loss: 1431.9718 - mae: 28.4234

KeyboardInterrupt: 

In [78]:
veggie_64.evaluate(X_test, y_test)

5/5 [==============================] - 2s 346ms/step - loss: 1050.1570 - mae: 25.6483
MAE: 25.64825439453125


(1050.156982421875, 25.64825439453125)

In [58]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_m = VeggieVision(base_model)

In [62]:
veggie_m.fit(X_train,y_train, learning_rate=1e-4, fine_tune=True, unfreeze_layers=10)

Epoch 1/20
15/15 [==============================] - 14s 767ms/step - loss: 14488.6465 - mae: 31.4260 - val_loss: 893.0674 - val_mae: 23.2445
Epoch 2/20
15/15 [==============================] - 10s 678ms/step - loss: 14175.6270 - mae: 31.1732 - val_loss: 889.7054 - val_mae: 23.2394
Epoch 3/20
15/15 [==============================] - 11s 703ms/step - loss: 14247.4004 - mae: 31.0830 - val_loss: 887.4605 - val_mae: 23.1640
Epoch 4/20
15/15 [==============================] - 11s 717ms/step - loss: 14239.5967 - mae: 29.4767 - val_loss: 885.1273 - val_mae: 23.0998
Epoch 5/20
15/15 [==============================] - 10s 666ms/step - loss: 14077.9111 - mae: 31.1282 - val_loss: 898.7690 - val_mae: 23.1130
Epoch 6/20
15/15 [==============================] - 10s 661ms/step - loss: 14146.6514 - mae: 31.1247 - val_loss: 885.8238 - val_mae: 23.1245
Epoch 7/20
15/15 [==============================] - 10s 666ms/step - loss: 14207.0605 - mae: 30.1597 - val_loss: 887.5978 - val_mae: 23.1143
Epoch 8/20
15

KeyboardInterrupt: 

In [64]:
veggie_m.evaluate(X_test, y_test)

5/5 [==============================] - 2s 387ms/step - loss: 1031.4767 - mae: 25.0926
MAE: 25.09260368347168


(1031.4766845703125, 25.09260368347168)

In [46]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_s = VeggieVision(base_model)

In [52]:
veggie_s.fit(X_train,y_train, learning_rate=1e-4, fine_tune= True, unfreeze_layers = 20)

Epoch 1/20
15/15 [==============================] - 14s 754ms/step - loss: 14639.6602 - mae: 33.4506 - val_loss: 902.5758 - val_mae: 23.6747
Epoch 2/20
15/15 [==============================] - 11s 696ms/step - loss: 14578.1582 - mae: 32.5870 - val_loss: 897.1644 - val_mae: 23.7533
Epoch 3/20
15/15 [==============================] - 11s 694ms/step - loss: 14083.1924 - mae: 32.0229 - val_loss: 897.5644 - val_mae: 23.6540
Epoch 4/20
 5/15 [=========>....................] - ETA: 6s - loss: 39467.5781 - mae: 41.2905

KeyboardInterrupt: 

In [53]:
veggie_s.evaluate(X_test, y_test)

5/5 [==============================] - 3s 478ms/step - loss: 1136.8997 - mae: 27.0319
MAE: 27.0318603515625


(1136.899658203125, 27.0318603515625)

In [34]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie = VeggieVision(base_model)

In [43]:
veggie.fit(X_train,y_train, learning_rate=1e-5, fine_tune=True, unfreeze_layers=20)

Epoch 1/20
15/15 [==============================] - 14s 751ms/step - loss: 12042.1787 - mae: 25.1193 - val_loss: 701.8392 - val_mae: 19.1236
Epoch 2/20
15/15 [==============================] - 11s 723ms/step - loss: 12968.1963 - mae: 25.5977 - val_loss: 701.6108 - val_mae: 19.1069
Epoch 3/20
15/15 [==============================] - 11s 722ms/step - loss: 13183.4004 - mae: 25.0411 - val_loss: 703.7874 - val_mae: 19.1098
Epoch 4/20
 5/15 [=========>....................] - ETA: 6s - loss: 825.9924 - mae: 21.1489

KeyboardInterrupt: 

In [40]:
veggie_saved = veggie

In [66]:
veggie_saved.model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 50176)             0         
                                                                 
 dense_17 (Dense)            (None, 128)               6422656   
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 1)                 129       
                                                                 
Total params: 9,651,649
Trainable params: 6,422,785
Non-trainable params: 3,228,864
____________________________________

In [37]:
veggie.evaluate(X_test, y_test)

5/5 [==============================] - 2s 440ms/step - loss: 600.7783 - mae: 17.8243
MAE: 17.82425880432129


(600.7782592773438, 17.82425880432129)

In [39]:
veggie.model.save("../models/small.h5")

In [63]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_vision_na = VeggieVision(base_model)

In [68]:
veggie_vision_na.fit(X,y, learning_rate=1e-5, fine_tune = True, unfreeze_layers = 10)

Epoch 1/20
19/19 [==============================] - 13s 605ms/step - loss: 498.8049 - mae: 5.9364 - val_loss: 470.2240 - val_mae: 15.6780
Epoch 2/20
 6/19 [========>.....................] - ETA: 6s - loss: 34.9182 - mae: 4.2940

KeyboardInterrupt: 

In [69]:
veggie_vision_na.evaluate(X,y)

23/23 [==============================] - 10s 414ms/step - loss: 921.2171 - mae: 8.2444
MAE: 8.244410514831543


(921.2171020507812, 8.244410514831543)

In [52]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_vision_ss = VeggieVision(base_model)

In [59]:
veggie_vision_ss.fit(X, y, learning_rate = 1e-5, fine_tune= True, unfreeze_layers=20)

Epoch 1/20
19/19 [==============================] - 15s 670ms/step - loss: 7550.2173 - mae: 19.9393 - val_loss: 582.4310 - val_mae: 16.2638
Epoch 2/20
19/19 [==============================] - 12s 649ms/step - loss: 7161.7163 - mae: 18.6796 - val_loss: 587.1320 - val_mae: 16.2770
Epoch 3/20
19/19 [==============================] - 13s 659ms/step - loss: 7014.6958 - mae: 19.0718 - val_loss: 587.1218 - val_mae: 16.2762
Epoch 4/20
19/19 [==============================] - 13s 654ms/step - loss: 6575.5649 - mae: 18.2659 - val_loss: 593.1298 - val_mae: 16.3183
Epoch 5/20


KeyboardInterrupt: 

In [60]:
veggie_vision_ss.evaluate(X,y)

23/23 [==============================] - 9s 399ms/step - loss: 5462.5327 - mae: 18.7492
MAE: 18.749235153198242


(5462.53271484375, 18.749235153198242)

In [41]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_vision_s = VeggieVision(base_model)

In [48]:
veggie_vision_s.fit(X, y, learning_rate = 1e-5, fine_tune=True, unfreeze_layers=10)

Epoch 1/20
19/19 [==============================] - 15s 726ms/step - loss: 4477.8740 - mae: 17.9785 - val_loss: 606.2526 - val_mae: 16.4656
Epoch 2/20
19/19 [==============================] - 13s 693ms/step - loss: 3254.8923 - mae: 17.7159 - val_loss: 599.2114 - val_mae: 16.2832
Epoch 3/20
19/19 [==============================] - 13s 686ms/step - loss: 1962.2288 - mae: 16.9120 - val_loss: 602.3713 - val_mae: 16.2894
Epoch 4/20
19/19 [==============================] - 13s 692ms/step - loss: 1780.1501 - mae: 16.4770 - val_loss: 606.3964 - val_mae: 16.4332
Epoch 5/20
19/19 [==============================] - 14s 718ms/step - loss: 3105.3254 - mae: 17.9044 - val_loss: 613.1610 - val_mae: 16.4559
Epoch 6/20
19/19 [==============================] - 13s 686ms/step - loss: 9133.3750 - mae: 19.9749 - val_loss: 612.0906 - val_mae: 16.4585
Epoch 7/20
19/19 [==============================] - 14s 732ms/step - loss: 2095.4500 - mae: 16.6710 - val_loss: 609.6676 - val_mae: 16.5089
Epoch 8/20
 1/19 [>.

KeyboardInterrupt: 

In [49]:
veggie_vision_s.evaluate(X, y)

23/23 [==============================] - 10s 418ms/step - loss: 2640.9199 - mae: 17.1886
MAE: 17.18863868713379


(2640.919921875, 17.18863868713379)

In [50]:
veggie_vision_s.model.save('../models/veggie_s.h5')

In [31]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_vision_v = VeggieVision(base_model)

In [38]:
veggie_vision_v.fit(X, y, learning_rate = 1e-6, fine_tune= True, unfreeze_layers=20)

Epoch 1/20
19/19 [==============================] - 17s 805ms/step - loss: 8627.0244 - mae: 20.3506 - val_loss: 799.5431 - val_mae: 19.1224
Epoch 2/20
19/19 [==============================] - 16s 809ms/step - loss: 5575.8301 - mae: 20.3383 - val_loss: 800.1811 - val_mae: 19.1332
Epoch 3/20
19/19 [==============================] - 15s 789ms/step - loss: 7160.3633 - mae: 22.0241 - val_loss: 799.5878 - val_mae: 19.1286
Epoch 4/20
 3/19 [===>..........................] - ETA: 12s - loss: 1222.0253 - mae: 21.7607

KeyboardInterrupt: 

In [39]:
veggie_vision_v.evaluate(X,y)

23/23 [==============================] - 10s 446ms/step - loss: 6253.5186 - mae: 20.0488
MAE: 20.048786163330078


(6253.5185546875, 20.048786163330078)

In [13]:
base_model = MobileNet(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_vision = VeggieVision(base_model)

In [23]:
veggie_vision.fit(X, y, learning_rate = 1e-4, fine_tune=True, unfreeze_layers = 20)

Epoch 1/20
19/19 [==============================] - 17s 802ms/step - loss: 7016.9761 - mae: 23.2022 - val_loss: 1088.1935 - val_mae: 19.1362
Epoch 2/20
19/19 [==============================] - 15s 772ms/step - loss: 8489.2461 - mae: 22.7899 - val_loss: 1052.4618 - val_mae: 18.8995
Epoch 3/20
19/19 [==============================] - 15s 756ms/step - loss: 3179.6577 - mae: 20.5613 - val_loss: 995.0499 - val_mae: 18.4289
Epoch 4/20
 2/19 [==>...........................] - ETA: 12s - loss: 787.0948 - mae: 20.2714

KeyboardInterrupt: 

In [24]:
veggie_vision.evaluate(X, y)

23/23 [==============================] - 10s 437ms/step - loss: 3853.2390 - mae: 21.8740
MAE: 21.8740177154541


(3853.239013671875, 21.8740177154541)